In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import random
import io
import re
from datetime import datetime, date, timedelta

#### **Scrape annonces**

In [3]:
url = 'https://www.seloger.com/list.htm?projects=2,5&types=2,1&natures=1,2,4&places=[{%22inseeCodes%22:[750115]}]&mandatorycommodities=0&enterprise=0&qsVersion=1.0&m=homepage_buy-redirection-search_results'
response = requests.get(url,headers= {'User-Agent': 'Safari/537.36'},timeout= random.randint(7, 14))
soup = BeautifulSoup(response.text,'html.parser')

#### **Scrape datas**

In [5]:
url = "https://www.seloger.com/annonces/achat/appartement/paris-15eme-75/emeriau-zola/231403269.htm?projects=2,5&types=2,1&natures=1,2,4&places=[{%22inseeCodes%22:[750115]}]&mandatorycommodities=0&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=L"
response = requests.get(url,headers= {'User-Agent': 'Safari/537.36'},timeout= random.randint(7, 14))
soup = BeautifulSoup(response.text,'html.parser')

In [6]:
data = {}

In [7]:
data['date'] = pd.Timestamp.now()

In [8]:
data['ref_code'] = re.search(r'\d{8}', url).group()

In [9]:
s = soup.find('div',{"class":"Summarystyled__Title-sc-1u9xobv-4 dbveQQ"}).text
if "Appartement" in s: 
    data['type'] = 'apartment'
elif "Maison" in s:
    data['type'] = 'house'
else:
    data['type'] = 'other'

In [10]:
s = soup.find('span', {'class': "global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"}).text
s = s.replace(" ", "").replace("€", "").strip()
data['price'] = s

In [11]:
s = soup.find('span', {'class' : "Localizationstyled__City-sc-gdkcr2-1 bgtLnh"}).text
data['zip_code'] = re.search(r'\((\d{5})\)', s).group(1)

In [12]:
s_list = [s.text for s in soup.find_all('div',{'class':"Tags__TagContainer-sc-edpl7u-0 EPxew"})]
data['room'] = s_list[0].split()[0]
data['bed'] = s_list[1].split()[0]
data['area'] = s_list[2].split()[0]

floor = s_list[3].split()[-1].split('/')

data['floor'] = floor[0]
data['max_floor'] = floor[1]

In [13]:
s = soup.find('div', {'class' : "Previewstyled__Grade-sc-k3u73o-6 ehFYCZ"}).text
data['dpe'] = s


In [14]:
data

{'date': Timestamp('2025-03-19 10:26:36.716860'),
 'ref_code': '23140326',
 'type': 'apartment',
 'price': '630000',
 'zip_code': '75015',
 'room': '2',
 'bed': '1',
 'area': '71',
 'floor': '20',
 'max_floor': '–',
 'dpe': 'E'}

In [ ]:
ref_code = data.pop('ref_code')
df = pd.DataFrame.from_dict({ref_code: data}, orient='columns')

In [17]:
df

,23140326
area,71
bed,1
date,2025-03-19 10:26:36.716860
dpe,E
floor,20
max_floor,–
price,630000
room,2
type,apartment
zip_code,75015


In [18]:
df.loc['date']

23140326    2025-03-19 10:26:36.716860
Name: date, dtype: object